# 🚀 State-of-the-Art AI Compiler Training (Google Colab)

**Free GPU Training** | Tesla T4 | ~2 hours

## Setup Instructions
1. Go to: https://colab.research.google.com/
2. File → Upload Notebook → Select this file
3. Runtime → Change runtime type → GPU → T4
4. Run all cells (Ctrl+F9)

## What Gets Trained
- ✅ Transformer Type Inference (GraphCodeBERT + GNN)
- ✅ Deep RL Strategy Agent (DQN + PPO)
- ✅ Meta-Learning (MAML)
- ✅ Multi-Agent System

## Free GPU Options
- **Colab**: 12-15 GB RAM, Tesla T4 GPU (free tier)
- **Colab Pro**: $9.99/month (faster GPUs, longer sessions)

In [ ]:
# 1️⃣ Check GPU
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# 2️⃣ Clone Your Repository from GitHub
!git clone https://github.com/soumitra-fr/Native-Python-Compiler.git
%cd Native-Python-Compiler

print("\n✅ Repository cloned successfully!")
print("📂 All your AI code is now available in Colab")

In [ ]:
# 3️⃣ Install Dependencies
!pip install -q torch transformers scikit-learn numpy pandas tqdm huggingface_hub accelerate sentencepiece protobuf

# Disable wandb (Weights & Biases) - not needed for training
import os
os.environ['WANDB_DISABLED'] = 'true'

print("✅ Dependencies installed")
print("✅ Wandb logging disabled (not needed)")

## 📚 Generate Training Data

In [ ]:
import sys
sys.path.insert(0, '/content/Native-Python-Compiler')

# Generate type inference training data
def generate_type_training_data(num_samples=2000):
    """Generate diverse type inference examples"""
    training_data = []
    
    patterns = [
        # Basic types
        ("x = {}", "x", "int", list(range(100))),
        ("y = {}", "y", "float", [i * 0.5 for i in range(100)]),
        ("s = '{}'", "s", "str", [f"string_{i}" for i in range(100)]),
        ("flag = {}", "flag", "bool", [True, False] * 50),
        
        # Collections
        ("lst = {}", "lst", "list", [[i, i+1, i+2] for i in range(100)]),
        ("d = {}", "d", "dict", [{f'k{i}': i} for i in range(100)]),
        ("t = {}", "t", "tuple", [(i, i+1) for i in range(100)]),
        ("s = {}", "s", "set", [{i, i+1, i+2} for i in range(100)]),
        
        # Complex patterns
        ("result = {} + 10", "result", "int", list(range(100))),
        ("total = sum({})", "total", "int", [[1,2,3], [4,5], [6,7,8,9]] * 33),
        ("items = [x*2 for x in {}]", "items", "list", [range(10), range(5)] * 50),
    ]
    
    for pattern, var, type_label, values in patterns:
        for value in values:
            code = pattern.format(repr(value))
            training_data.append({
                'code': code,
                'variable': var,
                'type': type_label
            })
    
    return training_data

print("📊 Generating training data...")
train_data = generate_type_training_data(2000)
val_data = generate_type_training_data(400)

print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Validation samples: {len(val_data)}")

## 🧠 Train Type Inference (GraphCodeBERT)

In [ ]:
from ai.transformer_type_inference import TransformerTypeInferenceEngine
import time
import torch

print("1️⃣  Training Transformer Type Inference...")
print("="*80)

# Clear GPU cache before training
torch.cuda.empty_cache()

engine = TransformerTypeInferenceEngine(device="cuda")

start_time = time.time()

engine.train(
    training_data=train_data,
    validation_data=val_data,
    epochs=3,  # Reduced from 10 to save memory
    batch_size=8,  # Reduced from 32 to fit in GPU memory
    learning_rate=2e-5,
    output_dir="./ai/models/type_inference_gpu"
)

training_time = time.time() - start_time
print(f"\n✅ Training complete in {training_time:.1f}s ({training_time/60:.1f} min)")

# Clear cache after training
torch.cuda.empty_cache()

# Test
test_code = "count = 42\nresult = count * 2"
pred = engine.predict(test_code, "count")
print(f"\n🧪 Test: {pred.type_name} (confidence: {pred.confidence:.2%})")

## 🤖 Train DQN Agent

In [ ]:
from ai.deep_rl_strategy import DeepRLStrategyAgent

print("2️⃣  Training DQN Strategy Agent...")
print("="*80)

dqn_agent = DeepRLStrategyAgent(device="cuda", batch_size=64, buffer_size=50000)

start_time = time.time()

dqn_agent.train(training_episodes=1000, eval_freq=100)  # More episodes on GPU

training_time = time.time() - start_time
print(f"\n✅ Training complete in {training_time:.1f}s ({training_time/60:.1f} min)")

dqn_agent.save("./ai/models/dqn_gpu")
print("💾 Model saved")

## 🎯 Train PPO Agent

In [ ]:
from ai.ppo_agent import PPOAgent

print("3️⃣  Training PPO Agent...")
print("="*80)

ppo_agent = PPOAgent(device="cuda", batch_size=64)

start_time = time.time()

ppo_agent.train(num_episodes=1000, steps_per_episode=100)

training_time = time.time() - start_time
print(f"\n✅ Training complete in {training_time:.1f}s ({training_time/60:.1f} min)")

ppo_agent.save("./ai/models/ppo_gpu")
print("💾 Model saved")

## 🔄 Train MAML Meta-Learning

In [ ]:
from ai.meta_learning import MAMLAgent, generate_synthetic_tasks

print("4️⃣  Training MAML Meta-Learning...")
print("="*80)

maml_agent = MAMLAgent(device="cuda")

train_tasks = generate_synthetic_tasks(100)
val_tasks = generate_synthetic_tasks(20)

start_time = time.time()

maml_agent.meta_train(train_tasks, val_tasks, num_iterations=500, tasks_per_batch=8)

training_time = time.time() - start_time
print(f"\n✅ Training complete in {training_time:.1f}s ({training_time/60:.1f} min)")

maml_agent.save("./ai/models/maml_gpu")
print("💾 Model saved")

## 💾 Download Trained Models

In [ ]:
# Package all trained models
!zip -r trained_models.zip ai/models/

print("\n✅ All models packaged!")
print("\n📥 Download 'trained_models.zip' from Colab Files panel (left sidebar)")
print("\n📝 Extract on your Mac:")
print("   unzip trained_models.zip")
print("   # Models will be in ai/models/ directory")

from google.colab import files
files.download('trained_models.zip')

## 📊 Training Summary

In [ ]:
import json

summary = {
    "device": "GPU (Colab)",
    "gpu_type": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A",
    "components_trained": [
        "Transformer Type Inference (GraphCodeBERT)",
        "DQN Strategy Agent (Dueling DQN + PER)",
        "PPO Agent (Actor-Critic + GAE)",
        "MAML Meta-Learning"
    ],
    "training_data_size": len(train_data),
    "validation_data_size": len(val_data),
    "status": "COMPLETE",
    "next_steps": [
        "Download trained_models.zip",
        "Extract to your Mac project",
        "Test with: python3 ai/test_trained_models.py",
        "Run benchmarks: python3 ai/benchmark_ai_components.py"
    ]
}

print(json.dumps(summary, indent=2))

with open('training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n💾 Summary saved to training_summary.json")